- 단계구분도: 경기도내 인구소멸비율

In [18]:
import json, folium
import numpy as np
import pandas as pd

- 경기도 시군구별 인구소멸비율 구하기

In [19]:
except_city = ['경기도 수원시', '경기도 고양시', '경기도 용인시', '경기도 성남시', '경기도 안산시', '경기도 안양시']

pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명', '행정구역명', '20~29세 (여)', '30~39세 (여)', '60~69세', '70~79세', '80~89세', '90~99세', '100세 이상', '총 인구수']]
pop = pop[((pop.행정구역구분명 == '시군') | (pop.행정구역구분명 == '구')) & ~(pop['행정구역명'].str.strip().isin(except_city))]
# pop = pop[((pop.행정구역구분명 == '시군') | (pop.행정구역구분명 == '구'))]
pop['인구소멸비율'] = (pop.iloc[:, 2:4].sum(axis=1) / pop.iloc[:, 4:9].sum(axis=1)).round(4)
pop = pop[['행정구역명', '총 인구수', '인구소멸비율']]
pop.head()

,행정구역명,총 인구수,인구소멸비율
1,경기도 가평군,62197,0.1976
9,경기도 고양시 덕양구,487874,0.5551
31,경기도 고양시 일산동구,296590,0.6047
44,경기도 고양시 일산서구,290738,0.5616
56,경기도 과천시,77775,0.6238


- geo data 읽기

In [20]:
fname = 'data/경기도_geo_simple.json' 
geo_data = json.load(open(fname, encoding='utf-8'))

- 인구 데이터와 geo 데이터간 인덱스 일치 시키기

In [21]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))


In [22]:
pop.head()

,행정구역명,총 인구수,인구소멸비율,시군구
1,경기도 가평군,62197,0.1976,가평군
9,경기도 고양시 덕양구,487874,0.5551,고양시덕양구
31,경기도 고양시 일산동구,296590,0.6047,고양시일산동구
44,경기도 고양시 일산서구,290738,0.5616,고양시일산서구
56,경기도 과천시,77775,0.6238,과천시


In [23]:
pop.set_index('시군구', inplace=True)
pop.head()

,행정구역명,총 인구수,인구소멸비율
시군구,,,
가평군,경기도 가평군,62197,0.1976
고양시덕양구,경기도 고양시 덕양구,487874,0.5551
고양시일산동구,경기도 고양시 일산동구,296590,0.6047
고양시일산서구,경기도 고양시 일산서구,290738,0.5616
과천시,경기도 과천시,77775,0.6238


- 단계 구분도

In [ ]:
import numpy as np

# 각 구별로 평균 위치 구하기
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            print(len(coord))
            geo = np.array(coord)
            
            if gu['properties']['name'] == '안산시단원구':
                # print(type(geo[0]))
                # print('end')
                # print(geo[0])
                # print(type([np.mean(geo[0][:,1]), np.mean(geo[0][:,0])]))
                gu_dict[gu['properties']['name']] = [np.mean(geo[0][:,1]), np.mean(geo[0][:,0])]
            else:
                # print(geo)
                gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
                # print([np.mean(geo[:,1]), np.mean(geo[:,0])])
    
    return gu_dict

gu_dict = get_text_location(geo_data)

In [25]:
smap = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')

folium.Choropleth(
    geo_data=geo_data,
    data=pop.인구소멸비율,                  # 단계구분도로 보여줄 데이터
    columns=[pop.index, pop.인구소멸비율],  # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',                   # Colormap
    key_on='properties.name'               # 지도에서 조인할 항목
).add_to(smap)

for gu_name in pop.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20, 0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>')
    ).add_to(smap)


title_html='<h3 align="center" style="font-size:20px;">경기도 내 인구소멸비율</h3>'
smap.get_root().html.add_child(folium.Element(title_html))
smap